<a href="https://colab.research.google.com/github/octonion/notebooks/blob/main/hedging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Hedge a Futures Bet with Large Equity

Say we have an active futures bet with large equity, and we'd like to hedge against the risk of our team losing.

For example, we're holding a ticket to collect \$81000.

https://x.com/HouseMoney77/status/1906875727250690215

Since it comes up all the time, I'm writing an article outlining some suggestions for hedging future sports bets. A first step is defining what you're trying to do i.e. what you're trying to maximize. My analysis is from the perspective of Kelly betting i.e. log-utility.

Even if you aren't a Kelly bettor, it's an excellent framework for learning and understanding the decision-making process for hedging.

A little summary on hedging using futures. The analysis turned out to be easier than expected, assuming you're using the probabilities implied by the futures themselves. Unfortunately, vigorish tends to be very high with futures, but the critical value has a nice closed-form.

This allows you to very easily decide to hedge or not hedge, *assuming* you want to hedge using futures and implied probabilities.

An alternative suggestion is to hedge on a game-by-game basis, using moneyline bets. The critical point here is the root of a quadratic.

## Hedging on a Rolling Game Basis

You can hedge using the tourney futures, but the vigorish is quite high. As an alternative, consider a two-step process. You first hedge on the game vs Auburn. If Florida advances, you also hedge the championship game.

Here are the details.

With the Florida-Auburn game, consider the three mutually exclusive outcomes - Florida beats Auburn and wins the championship with probability $p_1$, Florida beats Auburn but loses the championship with probability $p_2$, Auburn beats Florida with probability $p_3$.

Furthermore, let your bankroll be normalized to $1$, your fixed payout for a Florida championship $P$, and your bankroll $B$. Let $x = P/B$, and Auburn win return $b$.

We first seek to maximize bankroll fraction $f$ of

$$L = p_1 \log(1 + x - f) + p_2 \log(1 - f)+ p_3 \log(1 + f b).$$

If I did the math correctly, this will be the unique root in $(0,1)$ for the quadratic

$$b f^2 + [p_1 + p_2 - b(1 + p_3(1+x) + p_2 x)] f + [p_3 b(1+x) - p_1 - p_2(1+x)] = 0.$$

Hedging the championship game if Florida advances is substantially simpler. This will be

$$f^* = \min\left(1, (1 - p)(1 + x) - \frac{p}{b}\right),$$

where $x$ is defined as before, $b$ is the return if Florida's opponent wins, and $p$ is the implied probability of Florida winning.

In [12]:
# Payout normalized to bankroll

x = 8.1/10.0

# Futures

auburn = 6.920
duke = 1.970
florida = 3.890
houston = 5.740

s = 1/auburn + 1/duke + 1/florida + 1/houston

p_auburn = 1/auburn / s
p_duke = 1/duke / s
p_florida = 1/florida / s
p_houston = 1/houston / s

# Florida-Auburn

fa = 1.680
af = 2.250

s_fa = 1/fa + 1/af
p_fa = 1/fa / s_fa
p_af = 1/af / s_fa

# Return if Auburn beats Florida
b = af-1

# Florida championship
p_1 = p_florida
p_1

# Florida beats Auburn, loses championship
p_2 = p_fa - p_florida
p_2

# Auburn beats Florida
p_3 = p_af
p_3

import numpy as np

# Coefficients of the quadratic
c_2 = b
c_1 = (p_1 + p_2 - b*(1 + p_3*(1+x) + p_2 * x))
c_0 = (p_3 *b*(1+x) - p_1 - p_2*(1+x))

coefficients = [c_2,c_1,c_0]

# The desired f* is the root in [0,1]
roots = np.roots(coefficients)
print(roots)

[1.52258457 0.06468556]


##Hedging with Championship Futures using scipy.minimize

Using log-utility, we want to maximize

$$p_1 \log(1 + x - f_2-f_3-f_4) + p_2 \log(1 + f_2 b_2-f_3-f_4) + p_3 \log(1 + f_3 b_3-f_2-f_4) + p_4 \log(1 + f_4 b_4-f_2-f_3),$$

where $0 <= p_1, p_2, p_3, p_4$, $p_1+p_2+p_3+p_4 = 1$, $0 \leq f_2, f_3, f_4$, $f_2+f_3+f_4 \leq 1$ and $x \geq 0$.

In [31]:
from scipy.optimize import minimize
import numpy as np
import time

# maximize p_1 * log(1 + x - f_2-f_3-f_4)
# + p_2 * log(1 + f_2 * b_2-f_3-f_4)
# + p_3 * np.log(1 + f_3 * b_3-f_2-f_4)
# + p_4 * np.log(1 + f_4 * b_4-f_2-f_3)

# where 0 <= p_1, p_2, p_3, p_4
# p_1+p_2+p_3+p_4 = 1
# 0 <= f_2, f_3, f_4
# f_2+f_3+f_4 <= 1
# x >= 0

# Payout normalized to bankroll

x = 8.1/10.0

# Futures

auburn = 6.920
duke = 1.970
florida = 3.890
houston = 5.740

s = 1/auburn + 1/duke + 1/florida + 1/houston

p_auburn = 1/auburn / s
p_duke = 1/duke / s
p_florida = 1/florida / s
p_houston = 1/houston / s

obj = lambda f: -(p_florida*np.log(1+x+b_florida*f[3]-f[0]-f[1]-f[2]) + p_auburn*np.log(1+f[0]*b_auburn-f[1]-f[2]-f[3]) + p_duke*np.log(1+f[1]*b_duke-f[0]-f[2]-f[3]) + p_houston*np.log(1+f[2]*b_houston-f[0]-f[1]-f[3]))

# Jacobian
obj_Jac = lambda f: Jacobian(lambda f: obj(f))(f).ravel()

# constraints

# >= 0

constraints = ({'type': 'ineq', 'fun': lambda f: 1-(f[3]+f[0]+f[1]+f[2])}
               )

# bounds

# 0 <= f[i] <= 1

bounds = ((0,1),
          (0,1),
          (0,1),
          (0,1)
          )

f0 = (0,0,0,0)

start_time = time.time()*1000
result = minimize(obj, f0, bounds=bounds, constraints=constraints)
end_time = time.time()*1000
print('\n',result)
print("Optimal value p*", result.fun)
print("Optimal f: auburn = ", result.x[0], " duke = ", result.x[1], " houston = ", result.x[2])
print(f"Total fraction bet = {sum(result.x)}")
# Should generally be 0
#print("Optimal var: florida = ", result.x[3])
print("exec time (ms): ", end_time - start_time)


  message: Optimization terminated successfully
 success: True
  status: 0
     fun: -0.1445699305792817
       x: [ 4.425e-02  1.554e-01  5.344e-02  1.235e-16]
     nit: 5
     jac: [ 8.428e-05  1.255e-04  5.754e-04  2.837e-01]
    nfev: 26
    njev: 5
Optimal value p* -0.1445699305792817
Optimal f: auburn =  0.04424580309572772  duke =  0.15544698058084944  houston =  0.053444486422711136
Total fraction bet = 0.25313727009928844
exec time (ms):  4.2255859375


##A Closed-Form for Hedging with Championship Futures

Note that in the theory of Kelly betting one edge case is 0 vigorish i.e. fair bets. In this case, we bet our entire portfolio, with each bet allocated the fraction p that we estimate is the probability of each outcome.

In our case, we're estimating the implied probabilities from the odds themselves. With 0 vigorish, we'd always win/lose exactly \$0, regardless of the outcome.

In our hedge, we're again estimating the probabilities from the odds themselves, so we're always betting an amount proportional to the implied odds. This means we only need to optimize based on a composite futures bet i.e. a team other than Florida winning the championship.

Since these probabilities were estimated from the odds, and we're betting proportional to the implied probabilities, every outcome other than Florida results in the same net result for us.

Thus, we can view this simply as a Florida / not Florida bet, determining how much to bet on not Florida. Once we know this, we assign bets proportional to applied probabilities. Very easy to analyze.

This simplified analysis seeks $0≤f≤1$ that maximizes

$$p \log(1+x-f)+(1-p)\log(1+f(1-p)v-f).$$

The critical value is

$$f^* = (1-p)(1+x)-pv(1-p)/(1-v(1-p)).$$

If $f^* > 0$, you hedge, betting $\min(1,f^*)$ spread proportionally over the futures by implied probability.

Finally, we can indeed now see that we don't hedge with a bankroll of \\$200000 because the vigorish for championship futures is so high. We would hedge with a bankroll of \$200000 with smaller vigorish more like moneyline's.

In [30]:
# Payout normalized to bankroll

x = 8.1/10.0

# Futures

auburn = 6.920
duke = 1.970
florida = 3.890
houston = 5.740

s = 1/auburn + 1/duke + 1/florida + 1/houston

p_auburn = 1/auburn / s
p_duke = 1/duke / s
p_florida = 1/florida / s
p_houston = 1/houston / s

b_auburn = auburn-1
b_duke = duke-1
b_florida = florida-1
b_houston = houston-1

f = (1-p_florida)*(1+x)-p_florida*s*(1-p_florida)/(1-s*(1-p_florida))
print(f"Total fraction bet = {f}")

s_f = p_auburn + p_duke + p_houston
print("Optimal f: auburn = ", p_auburn/s_f * f, " duke = ", p_duke/s_f * f, " houston = ", p_houston/s_f * f)

Total fraction bet = 0.25147413976445865
Optimal f: auburn =  0.0439773476690516  duke =  0.15447880501006958  houston =  0.05301798708533746
